In [ ]:
import os
import time
import datetime
# import pymysql
# import itertools

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
os.environ['WDM_LOG_LEVEL'] = '0'
YOUTUBE_CHANNEL = '날림v'

In [ ]:
def init_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")
    options.add_argument("--log-level=3")
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1280, 1024)
    return driver

In [ ]:
def infinite_scroll (driver, SCROLL_PAUSE_TIME):
   time.sleep(SCROLL_PAUSE_TIME)

   while True:
      last_height = driver.execute_script("return document.documentElement.scrollHeight")
      driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

      time.sleep(SCROLL_PAUSE_TIME)

      new_height = driver.execute_script("return document.documentElement.scrollHeight")
      if new_height == last_height:
         break
      last_height = new_height

In [ ]:
def get_BGM_lists(driver, YOUTUBE_CHANNEL, SCROLL_PAUSE_TIME=2):
    driver.get(f'https://www.youtube.com/c/{YOUTUBE_CHANNEL}/playlists?view=1&sort=dd&shelf_id=0')
    infinite_scroll(driver, SCROLL_PAUSE_TIME)

    playlists = driver.find_elements(By.CSS_SELECTOR, '#items > ytd-grid-playlist-renderer')
    BGM_lists = [x.find_element(By.CSS_SELECTOR, '#video-title') for x in playlists
                if any(word in x.find_element(By.CSS_SELECTOR, '#video-title').text for word in ['BGM'])]

    playlist_href = [x.get_attribute("href").split('=')[-1] for x in BGM_lists]
    playlist_title = [x.text for x in BGM_lists]
    BGM_lists = sorted(dict(zip(playlist_title, playlist_href)).items())

    return BGM_lists

In [ ]:
driver = init_driver()
BGM_lists = get_BGM_lists(driver, YOUTUBE_CHANNEL)
print(BGM_lists)

In [ ]:
def get_list_items(driver, BGM_lists, SCROLL_PAUSE_TIME=2):
    total = []
    for list in BGM_lists:
        driver.get('https://www.youtube.com/playlist?list='+list[1])
        infinite_scroll(driver, SCROLL_PAUSE_TIME)

        videos = driver.find_elements(By.CSS_SELECTOR, '#meta')[:-1]
        rows = [(x.text.split("\n")[0],
                x.text.split("\n")[1],
                x.find_element(By.CSS_SELECTOR, '#video-title').get_attribute("href").split('&')[0].split('=')[1])
                for x in videos]
        total.extend(rows)
    return total

In [ ]:
total = get_list_items(driver, [BGM_lists[0]])
print(total)

In [ ]:
videos = driver.find_elements(By.CSS_SELECTOR, '#meta')[:-1]
print([x.find_element(By.CSS_SELECTOR, '#video-title').get_attribute("href").split('&')[0].split('=')[1]
        for x in videos])